In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layers
from trainertf_mod import Trainer

C:\Users\nicor\anaconda3\envs\tensorflow-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#data import
data_path_p= "./data/ben"
data_path_a= "./data/bena"
data_path_n= "./data/celeb2"
img_height = 256
img_width = 256
batch_size = 2

In [5]:
ds_pos = tf.keras.preprocessing.image_dataset_from_directory(
    data_path_p,
    labels=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width), #reshape if not thsi size
    shuffle=True, #randomize order
    seed=123,
)

ValueError: `labels` argument should be a list/tuple of integer labels, of the same size as the number of image files in the target directory. If you wish to infer the labels from the subdirectory names in the target directory, pass `labels="inferred"`. If you wish to get a dataset that only contains images (no labels), pass `label_mode=None`.

In [6]:
ds_anc = tf.keras.preprocessing.image_dataset_from_directory(
    data_path_a,
    labels=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width), #reshape if not thsi size
    shuffle=True, #randomize order
    seed=123,
)

ds_neg = tf.keras.preprocessing.image_dataset_from_directory(
    data_path_n,
    labels=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width), #reshape if not thsi size
    shuffle=True, #randomize order
    seed=123,
)

ValueError: `labels` argument should be a list/tuple of integer labels, of the same size as the number of image files in the target directory. If you wish to infer the labels from the subdirectory names in the target directory, pass `labels="inferred"`. If you wish to get a dataset that only contains images (no labels), pass `label_mode=None`.

In [7]:
def preproc(img):
    img = tf.image.resize(img, (224,224))
    #img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    #img = img[None, ...]
    print(img)
    return img
ds_p = ds_pos.map(preproc)
for x in ds_p.take(1):
    print('Image --> ', x.shape)

NameError: name 'ds_pos' is not defined

In [6]:
ds_n = ds_neg.map(preproc)
dataset = tf.data.Dataset.zip((ds_p, ds_p, ds_n))
dataset = dataset.shuffle(buffer_size=1024)
print(dataset)
for epoch in range(3):
    print(f'epoch {epoch}:')
    for step, X in enumerate(dataset):
        a , p , n = X

Tensor("resize/ResizeBilinear:0", shape=(None, 224, 224, 3), dtype=float32)
<ShuffleDataset shapes: ((None, 224, 224, 3), (None, 224, 224, 3), (None, 224, 224, 3)), types: (tf.float32, tf.float32, tf.float32)>
epoch 0:
epoch 1:
epoch 2:


In [7]:
for x, y in ds_anc.take(1):
    print('Image --> ', x.shape, 'Label --> ',  y.shape)

Image -->  (256, 256, 3) Label -->  (256, 256, 3)


In [8]:
class MyTrainer(Trainer):
    
    def metrics(self, L, X, Y_pred, Y_true):
        Y_pred_ndx = tf.argmax(Y_pred, axis=-1)
        acc = 0
        #acc = tf.reduce_mean(tf.cast(Y_pred_ndx == Y_true, tf.float32))
        return {'loss': L, 'acc': acc}

    def train_report(self, epoch, step):
        print(f"    {epoch:03d}/{step:05d}:  loss {self.loss:6.4f}, acc (avg) {self.avg_acc:6.4f}")

    def test_report(self, epoch, step):
        print('-' * 50)
        print(f"> {epoch:03d}/{step + 1:05d}:  loss {self.test_loss:6.4f}, acc {self.test_acc:6.4f}")
        print('-' * 50)

In [39]:
class Embedding(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.in_shape = (224,224,3)             
        self.mnet_mdl = tf.keras.applications.MobileNetV2(input_shape=self.in_shape, include_top=False, weights='imagenet')
        for l in self.mnet_mdl.layers[:144]:
            l.trainable=False
        for l in self.mnet_mdl.layers[144:]:
            l.trainable=True
            
        self.global_ap = layers.GlobalAveragePooling2D()
        
        #--> maybe more layers needed for embedding
        #self.dense1 = layers.Dense(512, activation="relu")
        #self.norm1 = layers.BatchNormalization()

        # Create embeddings with Batch Normalization or Layer Normalization 
        self.dense1 = layers.Dense(256, activation="relu")
        self.norm1 = layers.BatchNormalization()
        self.dense2 = layers.Dense(256, activation="softmax")

    def call(self, x0, training=False):
        x1 = self.mnet_mdl(x0)
        x2 = self.global_ap(x1)
        x3 = self.dense1(x2)
        x4 = self.norm1(x3)
        x5 = self.dense2(x4)
        return x5
    
    def sample_loss_fn(self, X, Y_true, Y_pred):
        single_squared_loss = tf.square(Y_pred)
        return tf.reduce_sum(single_squared_loss)

    def loss_fn(self, X, Y_true, Y_pred):
        return tf.reduce_mean(self.sample_loss_fn(X, Y_true, Y_pred))

In [40]:
cfg = {
    'n_epochs':     3, 
    'batch_size':   batch_size, 
    'opt':          tf.optimizers.Adam(learning_rate=0.001),
    'train_ds':     ds_p,   
    'test_ds':      None,
}

model1 = Embedding()
tr = MyTrainer(model1, cfg)
model1.summary()
hist = tr.train()

Found 10 train samples.
Model: "embedding_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_14  (None, 1280)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               327936    
_________________________________________________________________
batch_normalization_14 (Batc (None, 256)               1024      
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
Total params: 2,652,736
Trainable params: 1,126,720
Non-trainable params: 1,526,016
_________________________________________________________________
epoch 0:
    000/00000:  loss 0.0134, acc (avg) 0.0000
epoch 1:
    001/00000:

In [41]:
class TripletLayer(layers.Layer):
    def __init__(self, alph=0.5):
        super(TripletLayer, self).__init__()
    
    def call(self, inputs):
        anchor,positive,negative = inputs
        pos_distance = tf.math.reduce_sum(tf.math.square(anchor - positive), axis=1)
        neg_distance = tf.math.reduce_sum(tf.math.square(anchor - negative), axis=1)    
        return (pos_distance, neg_distance)

In [42]:
class FaceEmbedding(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.in_shape = (256,256,3)
        self.embedding = Embedding()
        self.triplet_l = TripletLayer()
        
    def call(self,X):
        #Embeddings for positive, anchor and negative
        anchor_input , positive_input , negative_input = X
        #print(positive_input)
        x_p = self.embedding(positive_input)
        x_a = self.embedding(anchor_input)
        x_n = self.embedding(negative_input)
        X_res = x_a , x_p , x_n
        #Tripplet loss
        x_los = self.triplet_l(X_res)
        return x_los

In [43]:
class ModelCompl(tf.keras.Model):
    def __init__(self, face_embedding):
        super().__init__()
        self.face_embeddings_mdl = face_embedding
        
    def call(self, X):
        return self.face_embeddings_mdl(X)
    
    def sample_loss_fn(self, embeddings):
        ap_distance, an_distance = embeddings
        loss = ap_distance - an_distance
        return loss
    
    def loss_fn(self, X, Y_true, Y_pred):
        return tf.math.reduce_max(self.sample_loss_fn(Y_pred) + 0.2)

In [46]:
cfg = {
    'n_epochs':     10, 
    'batch_size':   batch_size, 
    'opt':          tf.optimizers.Adam(learning_rate=0.001),
    'train_ds':     dataset,   
    'test_ds':      None,
}

embeding_1 = FaceEmbedding()
model_cpl = ModelCompl(embeding_1)
tr = MyTrainer(model_cpl, cfg)
#model_cpl.compile(tf.optimizers.Adam(learning_rate=0.001))

hst = tr.train()
model_cpl.summary()

Found 10 train samples.
epoch 0:
    000/00000:  loss 0.2021, acc (avg) 0.0000
epoch 1:
    001/00000:  loss 0.1999, acc (avg) 0.0000
epoch 2:
    002/00000:  loss 0.1983, acc (avg) 0.0000
epoch 3:
    003/00000:  loss 0.1984, acc (avg) 0.0000
epoch 4:
    004/00000:  loss 0.1987, acc (avg) 0.0000
epoch 5:
    005/00000:  loss 0.1956, acc (avg) 0.0000
epoch 6:
    006/00000:  loss 0.1934, acc (avg) 0.0000
epoch 7:
    007/00000:  loss 0.2030, acc (avg) 0.0000
epoch 8:
    008/00000:  loss 0.1285, acc (avg) 0.0000
epoch 9:
    009/00000:  loss 0.1342, acc (avg) 0.0000
Model: "model_compl_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
face_embedding_11 (FaceEmbed multiple                  2652736   
Total params: 2,652,736
Trainable params: 1,126,720
Non-trainable params: 1,526,016
_________________________________________________________________


In [45]:
class FaceEmbedding(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.in_shape = (256,256,3)           
        self.embedding = Embedding()
        self.triplet_l = TripletLayer()

    def call(self,X):
        #Embeddings for positive, anchor and negative
        anchor_input , positive_input , negative_input = X
        x_p = self.embedding(positive_input)
        x_a = self.embedding(anchor_input)
        x_n = self.embedding(negative_input)
        X_res = x_a , x_p , x_n
        #Tripplet loss
        x_los = self.triplet_l(X_res)
        return x_los
    
    def sample_loss_fn(self, embeddings):
        ap_distance, an_distance = embeddings
        loss = ap_distance - an_distance
        return loss
    
    def loss_fn(self, embeddings):
        return tf.math.reduce_max(sample_loss_fn(embeddings) + 0.2)

In [14]:
class MyTrainer(Trainer):
    def metrics(self, L, X, Y_pred, Y_true):
        #Y_pred_ndx = tf.argmax(Y_pred, axis=-1)
        #acc = tf.reduce_mean(tf.cast(Y_pred_ndx == Y_true, tf.float32))
        
        return {'loss': L}

    def train_report(self, epoch, step):
        print(f"    {epoch:03d}/{step:05d}:  loss {self.loss:6.4f}, acc (avg) {self.avg_acc:6.4f}")

    def test_report(self, epoch, step):
        print('-' * 50)
        print(f"> {epoch:03d}/{step + 1:05d}:  loss {self.test_loss:6.4f}, acc {self.test_acc:6.4f}")
        print('-' * 50)

In [15]:
model1 = FaceEmbedding()

In [16]:
cfg = {
    'n_epochs':     3,
    'batch_size':   batch_size,
    'opt':          tf.optimizers.Adam(learning_rate=0.001),
    'train_ds':     dataset,
    'test_ds':      dataset,
    'report_steps': 1,
}

In [18]:
tr = MyTrainer(model1, cfg)
model1.summary()

Found 10 train samples.
Found 10 test samples.


ValueError: You cannot build your model by calling `build` if your layers do not support float type inputs. Instead, in order to instantiate and build your model, `call` your model on real tensor data (of the correct dtype).